In [2]:
import pandas as pd
import psycopg2
from psycopg2 import sql

In [3]:
#load the csv file to pandas dataframe
df = pd.read_csv('covid.csv')

#display the details in a table
df.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [4]:
#check for the missing values
missing_values =df.isnull().sum()
data_types = df.dtypes

print(missing_values)
print(data_types)

df.head()


Country/Region            0
Confirmed                 0
Deaths                    0
Recovered                 0
Active                    0
New cases                 0
New deaths                0
New recovered             0
Deaths / 100 Cases        0
Recovered / 100 Cases     0
Deaths / 100 Recovered    0
Confirmed last week       0
1 week change             0
1 week % increase         0
WHO Region                0
dtype: int64
Country/Region             object
Confirmed                   int64
Deaths                      int64
Recovered                   int64
Active                      int64
New cases                   int64
New deaths                  int64
New recovered               int64
Deaths / 100 Cases        float64
Recovered / 100 Cases     float64
Deaths / 100 Recovered    float64
Confirmed last week         int64
1 week change               int64
1 week % increase         float64
WHO Region                 object
dtype: object


,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [5]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('covid.csv')  # Replace 'covid_data.csv' with your actual CSV file

# Adjust column names to match the database structure
df.columns = [
    'country_region', 'confirmed', 'deaths', 'recovered', 'active', 
    'new_cases', 'new_deaths', 'new_recovered', 'deaths_per_100_cases', 
    'recovered_per_100_cases', 'deaths_per_100_recovered', 
    'confirmed_last_week', 'one_week_change', 'one_week_percent_increase', 
    'who_region'
]

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="covid",  # Replace with your database name
    user="postgres",   # Replace with your username
    password="KARU55bime22",  # Replace with your password
    host="localhost",  # Replace with your PostgreSQL host
    port="5432"        # Default PostgreSQL port
)

# Create a cursor object
cursor = conn.cursor()

# Create the covid_data table
create_table_query = """
CREATE TABLE IF NOT EXISTS covid_data (
    country_region VARCHAR(100),
    confirmed BIGINT,
    deaths BIGINT,
    recovered BIGINT,
    active BIGINT,
    new_cases BIGINT,
    new_deaths BIGINT,
    new_recovered BIGINT,
    deaths_per_100_cases FLOAT,
    recovered_per_100_cases FLOAT,
    deaths_per_100_recovered FLOAT,
    confirmed_last_week BIGINT,
    one_week_change BIGINT,
    one_week_percent_increase FLOAT,
    who_region VARCHAR(50)
);
"""
cursor.execute(create_table_query)
conn.commit()

# Insert data into the table
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO covid_data (
        country_region, confirmed, deaths, recovered, active, 
        new_cases, new_deaths, new_recovered, deaths_per_100_cases, 
        recovered_per_100_cases, deaths_per_100_recovered, 
        confirmed_last_week, one_week_change, one_week_percent_increase, 
        who_region
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, tuple(row))

conn.commit()

# Query the database to check the contents
select_query = "SELECT * FROM covid_data LIMIT 5;"
cursor.execute(select_query)
result = cursor.fetchall()

# Close the connection
cursor.close()
conn.close()

# Display the result
print(result)


[('Afghanistan', 36263, 1269, 25198, 9796, 106, 10, 18, 3.5, 69.49, 5.04, 35526, 737, 2.07, 'Eastern Mediterranean'), ('Albania', 4880, 144, 2745, 1991, 117, 6, 63, 2.95, 56.25, 5.25, 4171, 709, 17.0, 'Europe'), ('Algeria', 27973, 1163, 18837, 7973, 616, 8, 749, 4.16, 67.34, 6.17, 23691, 4282, 18.07, 'Africa'), ('Andorra', 907, 52, 803, 52, 10, 0, 0, 5.73, 88.53, 6.48, 884, 23, 2.6, 'Europe'), ('Angola', 950, 41, 242, 667, 18, 1, 0, 4.32, 25.47, 16.94, 749, 201, 26.84, 'Africa')]


In [26]:
# Extrat data using weather csv file

import pandas as pd
import psycopg2
from psycopg2 import sql

df = pd.read_csv('weather.csv')
df.head()

missing_values = df.isnull().sum()
print(missing_values)

data_types = df.dtypes
print(data_types)

#drop the missing value column
df.drop(columns=['Precip Type'], inplace=True)

# Check it droped
print(df.columns)

# Convert 'Date' column to datetime type
df['Formatted Date'] = pd.to_datetime(df['Formatted Date'])

df.columns = [
    'Formatted_Date', 'Summary', 'Temperature_C', 
    'Apparent_Temperature_C', 'Humidity', 'Wind_Speed_kmph', 
    'Wind_Bearing_degrees', 'Visibility_km', 'Cloud_Cover', 
    'Pressure_millibars', 'Daily_Summary'
]

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="weather",
    user="postgres",  
    password="KARU55bime22",  
    host="localhost",  
    port="5432" 
)

# Create a cursor object
cursor = conn.cursor()

# Create the weather_data table
create_table_query = """
CREATE TABLE IF NOT EXISTS weather_data (
    Formatted_Date DATE,
    Summary TEXT,
    Temperature_C FLOAT,
    Apparent_Temperature_C FLOAT,
    Humidity FLOAT,
    Wind_Speed_kmph FLOAT,
    Wind_Bearing_degrees INT,
    Visibility_km FLOAT,
    Cloud_Cover FLOAT,
    Pressure_millibars FLOAT,
    Daily_Summary TEXT
);
"""
cursor.execute(create_table_query)
conn.commit()

# Insert data into the table
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO weather_data (
        Formatted_Date, Summary, Temperature_C, 
        Apparent_Temperature_C, Humidity, Wind_Speed_kmph, 
        Wind_Bearing_degrees, Visibility_km, Cloud_Cover, 
        Pressure_millibars, Daily_Summary
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, tuple(row))

conn.commit()

# Query the database to check the contents
select_query = "SELECT * FROM weather_data LIMIT 5;"
cursor.execute(select_query)
result = cursor.fetchall()

# Close the connection
cursor.close()
conn.close()

# Display the result
print(result)



Formatted Date                0
Summary                       0
Precip Type                 517
Temperature (C)               0
Apparent Temperature (C)      0
Humidity                      0
Wind Speed (km/h)             0
Wind Bearing (degrees)        0
Visibility (km)               0
Loud Cover                    0
Pressure (millibars)          0
Daily Summary                 0
dtype: int64
Formatted Date               object
Summary                      object
Precip Type                  object
Temperature (C)             float64
Apparent Temperature (C)    float64
Humidity                    float64
Wind Speed (km/h)           float64
Wind Bearing (degrees)      float64
Visibility (km)             float64
Loud Cover                  float64
Pressure (millibars)        float64
Daily Summary                object
dtype: object
Index(['Formatted Date', 'Summary', 'Temperature (C)',
       'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)',
       'Wind Bearing (degrees)', '

C:\Users\win10\AppData\Local\Temp\ipykernel_9976\1561835385.py:23: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Formatted Date'] = pd.to_datetime(df['Formatted Date'])


[(datetime.date(2006, 4, 1), 'Partly Cloudy', 9.47222222222222, 7.388888888888887, 0.89, 14.1197, 251, 15.826300000000002, 0.0, 1015.13, 'Partly cloudy throughout the day.'), (datetime.date(2006, 4, 1), 'Partly Cloudy', 9.355555555555558, 7.227777777777776, 0.86, 14.2646, 259, 15.826300000000002, 0.0, 1015.63, 'Partly cloudy throughout the day.'), (datetime.date(2006, 4, 1), 'Mostly Cloudy', 9.377777777777778, 9.377777777777778, 0.89, 3.9284, 204, 14.9569, 0.0, 1015.94, 'Partly cloudy throughout the day.'), (datetime.date(2006, 4, 1), 'Partly Cloudy', 8.28888888888889, 5.944444444444446, 0.83, 14.1036, 269, 15.826300000000002, 0.0, 1016.41, 'Partly cloudy throughout the day.'), (datetime.date(2006, 4, 1), 'Mostly Cloudy', 8.755555555555553, 6.977777777777779, 0.83, 11.0446, 259, 15.826300000000002, 0.0, 1016.51, 'Partly cloudy throughout the day.')]
